In [1]:
from datasets import load_dataset

ds = load_dataset("togethercomputer/RedPajama-Data-V2",
                  name="sample",
                  partition="head_middle",
                  snapshots=["2023-06"],
                  languages=["en", "es"]) 

/home/robinysh/.conda/envs/cs/lib/python3.11/site-packages/datasets/load.py:1486: FutureWarning: The repository for togethercomputer/RedPajama-Data-V2 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/togethercomputer/RedPajama-Data-V2
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Loading dataset shards:   0%|          | 0/22 [00:00<?, ?it/s]

In [6]:
from pathlib import Path
from tqdm.notebook import tqdm
import json
from icecream import ic
save_dir = Path("/home/public/data/redpajamav2")

with (save_dir/"raw/en.txt").open('w', encoding='utf-8') as f_en, (save_dir/"raw/es.txt").open('w', encoding='utf-8') as f_es:
    files = {'en': f_en, 'es': f_es}
    data = ds['train']
    for content, meta in tqdm(zip(data['raw_content'], data['meta']), total=len(data)):
        lang = json.loads(meta)['language']
        if lang not in files:
            continue
        files[lang].write(content)


  0%|          | 0/1050391 [00:00<?, ?it/s]

In [7]:
!mkdir $save_dir/truncated
!cp $save_dir/raw/* $save_dir/truncated
!truncate $save_dir/truncated/en.txt --reference $save_dir/truncated/es.txt

mkdir: cannot create directory '/home/public/data/redpajamav2/truncated': File exists
cp: -r not specified; omitting directory '/home/public/data/redpajamav2/raw/truncated'


In [9]:
!echo $save_dir

/home/public/data/redpajamav2


In [13]:
!mkdir $save_dir/splitted
!cd $save_dir
!awk '{if(rand()<0.95) {print $0 > "en.train"} else {print $0 > "en.test"}}' /home/public/data/redpajamav2/truncated/en.txt
!awk '{if(rand()<0.95) {print $0 > "es.train"} else {print $0 > "es.test"}}' /home/public/data/redpajamav2/truncated/es.txt
!mv *.train *.test $save_dir/splitted

mkdir: cannot create directory '/home/public/data/redpajamav2/splitted': File exists
